In [ ]:
#!pip install trafilatura, bs4
"""
On my mac at least i needed to
#pip install -U charset_normalizer
#pip uninstall charset_normalizer -y
#!pip install lxml_html_clean
"""
import requests
import trafilatura
from bs4 import BeautifulSoup

def fetch_text(uri):
    return trafilatura.extract(trafilatura.fetch_url(uri))

"""the simplest possible text scrape"""
site = 'https://docs.fastht.ml/'
downloaded = trafilatura.fetch_url(site)
home_content = trafilatura.extract(downloaded)
soup = BeautifulSoup(downloaded, 'html.parser')
links = soup.find_all('a')
all_links = set([link.get('href') for link in links if link.get('href','-')[0]=='.'])
all_links = [f"{site}{l[2:]}" for l in all_links]
all_pages = {uri: fetch_text(uri) for uri in all_links }
for k,v in all_pages.items():
    print(k, len(v))
home_content

In [ ]:
"""do some lame chunking"""
def chunk(s, l=10000):
    for i in range(0, len(s), l):
        yield i, s[i:i + l] 
        
chunks = []
for uri, content in all_pages.items():
    for i, c in chunk(content):
        chunks.append({'id':f"{uri}_{i}", 'uri':uri, 'text':c})
        
chunks

## Create a lancedb dump of the data

In [ ]:
import pydantic,lancedb
from lancedb.pydantic import LanceModel, Vector
from pydantic import BaseModel
import numpy as np
from lancedb.embeddings import get_registry

"""connect to the db (file)"""
db = lancedb.connect('../docs_search/app/data/')


"""https://lancedb.github.io/lancedb/embeddings/#openai-embedding-function"""
func = get_registry().get("openai").create(name="text-embedding-ada-002")

class HelpContent(LanceModel):
    """a simple lance data model to wrap the embedding in IO"""
    id: str
    uri: str
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()
        
"""Create a data model for content"""

table = db.create_table("fast_html_help",  schema=HelpContent, mode="overwrite")

table.add( chunks)

tbl = db.open_table('fast_html_help')
tbl.to_pandas()

## Check the search basically works

In [ ]:
"""check table directly"""
tbl.search('Is there a JS Walkthrough').select(['text']).to_list()

In [ ]:
"""check config for the vector store wrapper in the server utils"""
from app.src.chat_utils import VectorStore
store = VectorStore()
store('Is there a JS Walkthrough')

In [ ]:
"""check the agent
1. calls the store
2. calls GPT (requires OPEN_AI key)
"""
from app.src.chat_utils import agent
from IPython.display import Markdown
Markdown(agent('what is Fast HTML',is_streaming=False))

In [ ]:
Markdown(agent('Please give me an example snippet FastHTML python program',is_streaming=False))